# Lazy loading in Napari for big Datasets

Sometimes the 3D rendering in Napari is really slow because of the size of the dataset, and is even worst if we have a big set of labels like the ones that came from cell segmentation in lightsheet datasets because we can find millions of objects that requires a 32-bits label image that can exceed the size of the raw dataset itself.

For this reason, we develop this Jupyter Notebook to facilitate the visualization of this images, using a dask array of our zarr images for the lazy loading of the image.

## Setting up dependencies

In [1]:
import time
import os
import glob
import dask.array as da
import numpy as np
import zarr as zr
from tifffile import imread, imwrite
from aicsimageio import AICSImage
from aicsimageio.readers import BioformatsReader

## Start the Dask dashboard

In [ ]:
from dask.distributed import Client

client = Client()
client

In this example, we assume that we are going to visualize a dataset together with the coresspondant labels, you can add more channels or datasets as you want. In Napari is better to  import each channel as independent layer. In this example we are going to visualize two nuclear factors, SOX9 and OCT4 with it's labels generated with Stardist. 

In [ ]:
Images = 'G:\Stardist_results'

In [ ]:
import napari
from napari.utils import nbscreenshot
#import nest_asyncio
#nest_asyncio.apply()

v = napari.Viewer()
for img in os.listdir(Images):
    short_name = os.path.splitext(img)
    
    
    
    if img.endswith("_lb.tif"):
        store = AICSImage(Images+"/"+img, reader=BioformatsReader)
        store.dask_data
        store.xarray_dask_data
        
        v.add_labels(store.dask_data, name='label '+str(img[0]))
        
        print(os.path.join(Images, img)+' is in Napari')
        
        continue
        
    else:
        store = AICSImage(Images+"/"+img, reader=BioformatsReader)
        store.dask_data
        store.xarray_dask_data
        
        v.add_image(store.dask_data, name='image '+str(img[0]), contrast_limits=[0,65535])
        
        print(os.path.join(Images, img)+' is in Napari')
        
        continue

## Load one by one

In [2]:
import napari
from napari.utils import nbscreenshot

v = napari.Viewer()

img = r"G:\Stardist_results\EH3574_Oct4.tif"
short_name = os.path.splitext(img)

storeim = AICSImage(img, reader=BioformatsReader)
image= storeim.dask_data
storeim.xarray_dask_data
        
v.add_image(image, name='label '+str(short_name[0]))
        
print(img+' is in Napari')
print(image.shape, image.size, image.dtype)

C:\Users\Linux\.conda\envs\bio11\lib\site-packages\napari_tools_menu\__init__.py:168: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


G:\Stardist_results\EH3574_Oct4.tif is in Napari
(1053, 1, 1, 7177, 4117) 31113737577 uint16


In [4]:
lbl = r"G:\Stardist_results\EH3574_Oct4_lb.tif"
short_namel = os.path.splitext(lbl)

storelb = AICSImage(lbl, reader=BioformatsReader)
labels = storelb.dask_data
storelb.xarray_dask_data
        
v.add_labels(labels, name='label '+str(short_namel[0]))
        
print(lbl+' is in Napari')
print(labels.shape, labels.size, labels.dtype)

G:\Stardist_results\EH3574_Oct4_lb.tif is in Napari
(1053, 1, 1, 7177, 4117) 1048966505 int32


exception calling callback for <Future at 0x228e7b6fdc0 state=finished raised java.lang.IllegalArgumentException>
Traceback (most recent call last):
  File "ImageReader.java", line 539, in loci.formats.ImageReader.getIndex
  File "FormatReader.java", line 1105, in loci.formats.FormatReader.getIndex
  File "FormatTools.java", line 408, in loci.formats.FormatTools.getIndex
  File "FormatTools.java", line 456, in loci.formats.FormatTools.getIndex
Exception: Java Exception

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\Linux\.conda\envs\bio11\lib\concurrent\futures\_base.py", line 330, in _invoke_callbacks
    callback(self)
  File "C:\Users\Linux\.conda\envs\bio11\lib\site-packages\napari\components\experimental\chunk\_pool.py", line 153, in _on_done
    request = self._get_request(future)
  File "C:\Users\Linux\.conda\envs\bio11\lib\site-packages\napari\components\experimental\chunk\_pool.py", line 189, in _get_r

In [ ]:
images = (r"D:\EHG\Gonad\Raw\19-48-26_EH3574_PCW106_5-COUPTFII_6-Oct4_7-Sox9_12x_1z_STITCHED.ome_C1.zarr",r"D:\EHG\Gonad\Raw\19-48-26_EH3574_PCW106_5-COUPTFII_6-Oct4_7-Sox9_12x_1z_STITCHED.ome_C0.zarr")

Oct4 = (zr.load(images[0]))
lazy_Oct4 = (lazy_load(images[0]))
lazy_Sox9 = (lazy_load(images[1]))

dask_Oct4 = da.from_delayed(lazy_Oct4, shape=Oct4.shape, dtype=Oct4.dtype)
dask_Sox9 = da.from_delayed(lazy_Sox9, shape=Oct4.shape, dtype=Oct4.dtype)

dask_Oct4.shape
dask_Sox9.shape

print(Oct4.shape)
dask_Oct4        

In [ ]:
dask_Sox9

In [ ]:
desired_chunksize = np.array([10,2,2])

In [ ]:
rechunked_Oct4 = dask_Oct4.rechunk(desired_chunksize)
rechunked_Oct4

In [ ]:
import napari
from napari.utils import nbscreenshot
import nest_asyncio
nest_asyncio.apply()

In [ ]:
v = napari.Viewer()
v.add_image(dask_Oct4, name="Oct4", contrast_limits=[0,1], cache=False)
v.add_image(dask_Sox9, name="Sox9", contrast_limits=[0,1], cache=False)

In [ ]:
labels = (r"G:\Stardist_results\LblImg_19-48-26_EH3574_PCW106_5-COUPTFII_6-Oct4_7-Sox9_12x_1z_STITCHED.ome_C1.tif1.zarr", r"G:\Stardist_results\LblImg_19-48-26_EH3574_PCW106_5-COUPTFII_6-Oct4_7-Sox9_12x_1z_STITCHED_C0_lbl.tif1.zarr")

Oct4l = (zr.load(labels[0]))
lazy_labels = [lazy_load(lbl) for lbl in labels]
dask_labels = [
    da.from_delayed(delayed_reader, shape=Oct4l.shape, dtype=Oct4l.dtype)
    for delayed_reader in lazy_labels
]

lstack = da.stack(dask_labels, axis=0)
lstack.shape

lstack

In [ ]:
import napari
from napari.utils import nbscreenshot

In [ ]:
v = napari.Viewer()
v.add_image(istack, contrast_limits=[0,2000], multiscale=False)